In [2]:
import os
os.chdir("../")

# UmojaHack Africa 2022: African Snake Antivenom Binding Challenge Starter Notebook 🐍 🧬




The challenge dataset stems from a high-density peptide microarray experiment that aimed to address how cross-reactive 8 different commercially available snake antivenoms are and where in the toxin sequence the antibodies they contain bind the toxin (epitope).


You will need to predict the signal column generated by a given Toxin_Kmer and Antivenom. You can use any other column available in the test set to enhance your predictions or enrich your data. We also facilitate the protein prot_bert embeddings for each row

Any questions feel free to reach the authors.

Authors :  M Jedidi (meds.jedidi@instadeep.com) and Nicolas Lopez Carranza (n.lopezcarranza@instadeep.com)

The dataset has been provided by Tim Jenkins - DTU Bionengineering (tpaje@dtu.dk)

## Libraries

In [ ]:
!pip install -qU tensorboard numpy pandas torch sklearn tensorflow

     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 462 kB 46.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.


In [ ]:
%load_ext tensorboard

In [ ]:
import random

import numpy as np 
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn 
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import collections
from itertools import combinations, product

In [ ]:
torch.manual_seed(125)
random.seed(15546)
np.random.seed(166)

## Read data

In [ ]:
train_df = pd.read_csv("https://storage.googleapis.com/umojahack2022/train.csv")
test_df = pd.read_csv("https://storage.googleapis.com/umojahack2022/test.csv")

## Basic EDA

In [ ]:
test_df.tail()

,ID,Toxin_UniprotID,Kmer_Position_start,Kmer_Position_end,Antivenom,Toxin_Kmer,Genus,Species,ProteinFam,ProteinSubFam,ProteinSubSubFam
36535,P01405_VINS_Central_Africa_41,P01405,41,56,VINS_Central_Africa,PKKEIFRKSIHCCRSD,Dendroaspis,Dendroaspis_viridis,Snake_three_finger_toxin,Short_chain_subfamily,Orphan_group_XI_sub_subfamily
36536,P01405_VINS_Central_Africa_42,P01405,42,57,VINS_Central_Africa,KKEIFRKSIHCCRSDK,Dendroaspis,Dendroaspis_viridis,Snake_three_finger_toxin,Short_chain_subfamily,Orphan_group_XI_sub_subfamily
36537,P01405_VINS_Central_Africa_43,P01405,43,58,VINS_Central_Africa,KEIFRKSIHCCRSDKC,Dendroaspis,Dendroaspis_viridis,Snake_three_finger_toxin,Short_chain_subfamily,Orphan_group_XI_sub_subfamily
36538,P01405_VINS_Central_Africa_44,P01405,44,59,VINS_Central_Africa,EIFRKSIHCCRSDKCN,Dendroaspis,Dendroaspis_viridis,Snake_three_finger_toxin,Short_chain_subfamily,Orphan_group_XI_sub_subfamily
36539,P01405_VINS_Central_Africa_45,P01405,45,60,VINS_Central_Africa,IFRKSIHCCRSDKCNE,Dendroaspis,Dendroaspis_viridis,Snake_three_finger_toxin,Short_chain_subfamily,Orphan_group_XI_sub_subfamily


In [ ]:
train_df.tail()

,ID,Toxin_UniprotID,Kmer_Position_start,Kmer_Position_end,Antivenom,Toxin_Kmer,Signal,Genus,Species,ProteinFam,ProteinSubFam,ProteinSubSubFam
124112,M5BGY5_VINS_Central_Africa_92,M5BGY5,92,107,VINS_Central_Africa,DMNDYCTGTTPDCPRN,1.06,Atheris,Atheris_chlorechis,Disintegrin,Dimeric_disintegrin_subfamily,NaN
124113,M5BGY5_VINS_Central_Africa_93,M5BGY5,93,108,VINS_Central_Africa,MNDYCTGTTPDCPRNP,0.80,Atheris,Atheris_chlorechis,Disintegrin,Dimeric_disintegrin_subfamily,NaN
124114,M5BGY5_VINS_Central_Africa_94,M5BGY5,94,109,VINS_Central_Africa,NDYCTGTTPDCPRNPY,0.43,Atheris,Atheris_chlorechis,Disintegrin,Dimeric_disintegrin_subfamily,NaN
124115,M5BGY5_VINS_Central_Africa_95,M5BGY5,95,110,VINS_Central_Africa,DYCTGTTPDCPRNPYK,1.46,Atheris,Atheris_chlorechis,Disintegrin,Dimeric_disintegrin_subfamily,NaN
124116,M5BGY5_VINS_Central_Africa_96,M5BGY5,96,111,VINS_Central_Africa,YCTGTTPDCPRNPYKD,1.47,Atheris,Atheris_chlorechis,Disintegrin,Dimeric_disintegrin_subfamily,NaN


Each row in the dataset represents a k-mer (16 amino acid sequence within the toxin) and it has a signal column coming from the high-density peptide microarray experiment. The dataframe has the following columns :
```
ID: Unique identifier for each row 
Toxin_UniprotID: Identifier for a specific toxin sequence in the Uniprot Database
Kmer_Position_start: The start position in the toxin global sequence of the Kmer_Position_end: The end  position in the toxin global sequence a given k-mer 
Antivenom: Name of the antivenom tested in the high-density peptide microarray experiment
Toxin_Kmer: String of 16 amino acids (16-mer, K=16) from a given toxin sequence
Signal: (target) The output of the experiment. A proxy for antivenom activity.
Genus: Genus of snake the toxin stems from, e.g. Naja (cobra)
Species: Species of snake the toxin originates from e.g. Naja nigricollis (Black-necked spitting cobra)
ProteinFam: Toxin protein family, e.g. three finger toxin (3FTx)
ProteinSubFam: Toxin sub-family, e.g. cytotoxin (a type of 3FTx)
ProteinSubSubFam: Toxin sub-sub-family, e.g. cytotoxin IA (a type of cytotoxin)
```

We can use any of these colums to train our ML model.

For our model, we will use the `Antivenom` and the `Toxin_Kmer` and the `Kmer_Position_start` columns.

In [ ]:
# train_df.rename({"Toxin_K_mer":"Toxin_Kmer", "Position_start":"Kmer_Position_start", "Position_end":"Kmer_Position_end"}, axis=1, inplace=True)
# test_df.rename({"Toxin_K_mer":"Toxin_Kmer", "Position_start":"Kmer_Position_start", "Position_end":"Kmer_Position_end"}, axis=1, inplace=True)

In [ ]:
def get_seq_column_map(train, test, col):
    sequences = []
    for seq in train[col]:
        sequences.extend(list(seq))
    for seq in test[col]:
        sequences.extend(list(seq))
    unique = np.unique(sequences)
    return {k: v for k, v in zip(unique, range(len(unique)))}

def get_column_map(train, test, col):
    sequences = []
    unique_values = pd.concat([train[col], test[col]]).unique().tolist()
    return {k: v for k, v in zip(unique_values, range(len(unique_values)))}

In [ ]:
train_df.select_dtypes("object").columns

Index(['ID', 'Toxin_UniprotID', 'Antivenom', 'Toxin_Kmer', 'Genus', 'Species',
       'ProteinFam', 'ProteinSubFam', 'ProteinSubSubFam'],
      dtype='object')

In [ ]:
amino_acid_map = get_seq_column_map(train_df, test_df, "Toxin_Kmer")
print("unique amino acid map",len(amino_acid_map))

antivenom_map = get_column_map(train_df, test_df, "Antivenom")
print("unique Antivenom map", len(antivenom_map))

genus_map = get_column_map(train_df, test_df, "Genus")
print("unique Genus map", len(genus_map))

species_map = get_column_map(train_df, test_df, "Species")
print("unique Species map", len(species_map))

proteinfam_map = get_column_map(train_df, test_df, "ProteinFam")
print("unique ProteinFam map", len(proteinfam_map))

unique amino acid map 20
unique Antivenom map 8
unique Genus map 8
unique Species map 32
unique ProteinFam map 17


In [ ]:
amino_acid_map

{'A': 0,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'K': 8,
 'L': 9,
 'M': 10,
 'N': 11,
 'P': 12,
 'Q': 13,
 'R': 14,
 'S': 15,
 'T': 16,
 'V': 17,
 'W': 18,
 'Y': 19}

In [ ]:
antivenom_map

{'Bioclone': 0,
 'ICP': 1,
 'Inosan': 2,
 'Premium': 3,
 'SAVP': 4,
 'Sanofi_Pasteur': 5,
 'VINS_Africa': 6,
 'VINS_Central_Africa': 7}

In [ ]:
genus_map

{'Aspidelaps': 0,
 'Atheris': 7,
 'Bitis': 5,
 'Dendroaspis': 6,
 'Dispholidus': 4,
 'Echis': 1,
 'Hemachatus': 3,
 'Naja': 2}

In [ ]:
species_map

{'Aspidelaps_scutatus': 0,
 'Atheris_chlorechis': 27,
 'Atheris_nitschei': 31,
 'Atheris_squamigera': 28,
 'Bitis_arietans': 19,
 'Bitis_caudalis': 29,
 'Bitis_gabonica': 20,
 'Bitis_nasicornis': 21,
 'Bitis_rhinoceros': 22,
 'Dendroaspis_angusticeps': 23,
 'Dendroaspis_jamesoni': 24,
 'Dendroaspis_polylepis': 25,
 'Dendroaspis_viridis': 26,
 'Dispholidus_typus': 18,
 'Echis_carinatus': 1,
 'Echis_coloratus': 2,
 'Echis_leucogaster': 3,
 'Echis_multisquamatus': 4,
 'Echis_ocellatus': 5,
 'Echis_pyramidum': 6,
 'Hemachatus_haemachatus': 17,
 'Naja_anchietae': 7,
 'Naja_annulata': 8,
 'Naja_annulifera': 9,
 'Naja_christyi': 30,
 'Naja_haje': 10,
 'Naja_melanoleuca': 11,
 'Naja_mossambica': 12,
 'Naja_nigricollis': 13,
 'Naja_nivea': 14,
 'Naja_oxiana': 15,
 'Naja_pallida': 16}

In [ ]:
proteinfam_map

{'AVIT_prokineticin': 15,
 'CRISP': 2,
 'Cystatin': 14,
 'Disintegrin': 3,
 'Flavin_monoamine_oxidase': 7,
 'Glycosyl_hydrolase_56': 10,
 'NGF_beta': 16,
 'Natriuretic_peptide': 13,
 'PDGF_VEGF_growth_factor': 11,
 'Peptidase_S1': 6,
 'Phospholipase_A2': 0,
 'Snaclec': 5,
 'Snake_three_finger_toxin': 1,
 'Snake_waprin': 8,
 'True_venom_lectin': 12,
 'Venom_Kunitz_type': 9,
 'Venom_metalloproteinase_M12B': 4}

In [ ]:
def one_gram(kmer):
  count_acides_map = collections.Counter(kmer)
  default_acide_map = {k:0 for k in amino_acid_map.keys()}
  count_acides_map.update(default_acide_map)
  f_i_acides = {k:v for k, v in dict(count_acides_map).items()}
  return list(f_i_acides.values())

def get_one_grams(df):
  names = list(amino_acid_map.keys())
  toxin_kmer_preprocessed = [one_gram(x) for x in tqdm(df["Toxin_Kmer"].values.tolist())]
  toxin_kmer_preprocessed = pd.DataFrame(data = np.array(toxin_kmer_preprocessed), columns = names)
  return toxin_kmer_preprocessed



In [ ]:
def bi_gram(kmer, r):
  kmer = train_df["Toxin_Kmer"][0]
  bigram_count = collections.Counter([kmer[i:i+2] for i in range(16) if len(kmer[i:i+2])==r])
  default_map = {k:0 for k in ["".join(x) for x in list(product(amino_acid_map.keys(), repeat=r))]}
  bigram_count.update(default_map)
  f_i_acides = {k:v for k, v in dict(bigram_count).items()}  
  return list(f_i_acides.values())

def get_bi_grams(df, r):
  names = ["".join(x) for x in list(product(amino_acid_map.keys(), repeat=r))]
  toxin_kmer_preprocessed = [bi_gram(x, r) for x in tqdm(df["Toxin_Kmer"].values.tolist())]
  toxin_kmer_preprocessed = pd.DataFrame(data = np.array(toxin_kmer_preprocessed), columns = names)
  return toxin_kmer_preprocessed

In [ ]:
train_add = pd.concat([get_one_grams(train_df), get_bi_grams(train_df, 2)], axis = 1)
test_add = pd.concat([get_one_grams(test_df), get_bi_grams(test_df, 2)], axis = 1)

100%|██████████| 36540/36540 [00:12<00:00, 2962.65it/s]


In [ ]:
train_add.shape, test_add.shape

((124117, 420), (36540, 420))

In [ ]:
print(list(train_add.var()[train_add.var()==0].index))

['S', 'T', 'V', 'W', 'Y', 'AS', 'AT', 'AV', 'AW', 'AY', 'CA', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL', 'CM', 'CN', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CV', 'CW', 'CY', 'DA', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DK', 'DL', 'DM', 'DN', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DV', 'DW', 'DY', 'EA', 'EC', 'ED', 'EE', 'EF', 'EG', 'EH', 'EI', 'EK', 'EL', 'EM', 'EN', 'EP', 'EQ', 'ER', 'ES', 'ET', 'EV', 'EW', 'EY', 'FA', 'FC', 'FD', 'FE', 'FF', 'FG', 'FH', 'FI', 'FK', 'FL', 'FM', 'FN', 'FP', 'FQ', 'FR', 'FS', 'FT', 'FV', 'FW', 'FY', 'GA', 'GC', 'GD', 'GE', 'GF', 'GG', 'GH', 'GI', 'GK', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR', 'GS', 'GT', 'GV', 'GW', 'GY', 'HA', 'HC', 'HD', 'HE', 'HF', 'HG', 'HH', 'HI', 'HK', 'HL', 'HM', 'HN', 'HP', 'HQ', 'HR', 'HS', 'HT', 'HV', 'HW', 'HY', 'IA', 'IC', 'ID', 'IE', 'IF', 'IG', 'IH', 'II', 'IK', 'IL', 'IM', 'IN', 'IP', 'IQ', 'IR', 'IS', 'IT', 'IV', 'IW', 'IY', 'KA', 'KC', 'KD', 'KE', 'KF', 'KG', 'KH', 'KI', 'KK', 'KL', 'KM', 'KN', 'KP', 'KQ', 'KR', 'KS', 'KT', 'K

In [ ]:
print(list(test_add.var()[test_add.var()==0].index))

['S', 'T', 'V', 'W', 'Y', 'AA', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AK', 'AL', 'AM', 'AN', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AV', 'AW', 'AY', 'CA', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL', 'CM', 'CN', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CV', 'CW', 'CY', 'DA', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DK', 'DL', 'DM', 'DN', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DV', 'DW', 'DY', 'EA', 'EC', 'ED', 'EE', 'EF', 'EG', 'EH', 'EI', 'EK', 'EL', 'EM', 'EN', 'EP', 'EQ', 'ER', 'ES', 'ET', 'EV', 'EW', 'EY', 'FA', 'FC', 'FD', 'FE', 'FF', 'FG', 'FH', 'FI', 'FK', 'FL', 'FM', 'FN', 'FP', 'FQ', 'FR', 'FS', 'FT', 'FV', 'FW', 'FY', 'GA', 'GC', 'GD', 'GE', 'GF', 'GG', 'GH', 'GI', 'GK', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR', 'GS', 'GT', 'GV', 'GW', 'GY', 'HA', 'HC', 'HD', 'HE', 'HF', 'HG', 'HH', 'HI', 'HK', 'HL', 'HM', 'HN', 'HP', 'HQ', 'HR', 'HS', 'HT', 'HV', 'HW', 'HY', 'IA', 'IC', 'ID', 'IE', 'IF', 'IG', 'IH', 'II', 'IK', 'IL', 'IM', 'IN', 'IP', 'IQ', 'IR', 'IS', 'IT', 'IV', 'IW', 'IY', 'KA', 'KC', 'K

In [ ]:
train_add.drop(list(test_add.var()[test_add.var()==0].index), axis=1, inplace=True)
test_add.drop(list(test_add.var()[test_add.var()==0].index), axis=1, inplace=True)

In [ ]:
train_add.tail()

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R
124112,3,1,2,1,2,3,1,2,1,0,0,0,0,0,0
124113,1,2,2,1,2,3,1,3,1,0,0,0,0,0,0
124114,2,2,2,2,3,1,3,1,0,0,0,0,0,0,0
124115,2,2,2,3,1,3,1,1,1,0,0,0,0,0,0
124116,2,2,3,1,3,2,1,1,1,0,0,0,0,0,0


In [ ]:
test_add.tail()

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R
36535,1,3,1,2,1,2,2,1,2,1,0,0,0,0,0
36536,4,1,2,1,2,2,1,2,1,0,0,0,0,0,0
36537,3,1,2,1,2,2,1,3,1,0,0,0,0,0,0
36538,1,2,1,2,2,2,1,3,1,1,0,0,0,0,0
36539,2,1,2,2,2,1,3,1,1,1,0,0,0,0,0


In [ ]:
train_add.shape, test_add.shape

((124117, 15), (36540, 15))

In [ ]:
additional_cols = train_add.columns.tolist()
additional_cols

['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R']

In [ ]:
train_add.to_csv("Data/train_add.csv", index=False)
test_add.to_csv("Data/test_add.csv", index=False)

In [ ]:
train_df = pd.concat([train_df, train_add], axis=1)
test_df = pd.concat([test_df, test_add], axis=1)

In [ ]:
def acid_processing(acid):

  toxin_map = {k:[0]*16 for k in amino_acid_map.keys()}

  for i, acide in enumerate(acid):
      toxin_map[acide][i] = 1

  return np.array(list(toxin_map.values())).ravel().tolist()

In [ ]:
names = ["".join([str(c) for c in x ])for x in list(product(list(amino_acid_map.keys()), range(16)))]

In [ ]:
def get_toxin_kmer(df):
  toxin_kmer_preprocessed = [acid_processing(x) for x in tqdm(df["Toxin_Kmer"].values.tolist())]
  toxin_kmer_preprocessed = pd.DataFrame(data = np.array(toxin_kmer_preprocessed), columns = names)
  return toxin_kmer_preprocessed

In [ ]:
train_df = pd.concat([get_toxin_kmer(train_df), train_df], axis=1)
test_df = pd.concat([get_toxin_kmer(test_df), test_df], axis=1)

100%|██████████| 36540/36540 [00:01<00:00, 18357.41it/s]


We will split the data into a training and a validation set

In [ ]:
UniprotID_train, UniprotID_val = train_test_split(train_df["Toxin_UniprotID"].unique(), test_size=0.2, random_state=20199)

train_split_df = train_df[train_df.Toxin_UniprotID.isin(UniprotID_train)]
val_split_df = train_df[train_df.Toxin_UniprotID.isin(UniprotID_val)]

print(f"train: {len(train_split_df)}, unique UniprotID {train_split_df.Toxin_UniprotID.nunique()}")
print(f"test: {len(val_split_df)}, unique UniprotID {val_split_df.Toxin_UniprotID.nunique()}")

train: 98854, unique UniprotID 189
test: 25263, unique UniprotID 48


We look at the GPU provided by Colab

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device : {device}")
torch.cuda.get_device_name()

device : cuda


'Tesla K80'

We convert our data into a torch `Dataset`.
All datasets that represent a map from keys to data samples should subclass
`Dataset`. All subclasses should overwrite `__getitem__`, supporting fetching a data sample for a given key:

In [ ]:
class AntivenomChallengeDataSet(Dataset):
    def __init__(
        self,
        amino_acid_map,
        antivenom_map,
        genus_map,
        species_map,
        proteinfam_map,
        data,
        is_train,
        label_name=None,
      ):
        self.amino_acid_map = amino_acid_map
        self.antivenom_map = antivenom_map
        self.genus_map = genus_map
        self.species_map = species_map
        self.proteinfam_map = proteinfam_map
        self.data = data
        self.is_train = is_train
        self.label_name = label_name

    def __len__(self):
        return len(self.data) 

    def __getitem__(self,idx):
        row = self.data.iloc[idx]
        kmer_seq = torch.as_tensor([self.amino_acid_map[e] for e in list(row["Toxin_Kmer"])])
        kmer_seq_sparsed = torch.as_tensor(list(row[names].values))
        antivenom = torch.as_tensor(self.antivenom_map[row["Antivenom"]])
        genus = torch.as_tensor(self.genus_map[row["Genus"]])
        species = torch.as_tensor(self.species_map[row["Species"]])
        proteinfam = torch.as_tensor(self.proteinfam_map[row["ProteinFam"]])
        position_start = torch.as_tensor(row["Kmer_Position_start"])
        position_end = torch.as_tensor(row["Kmer_Position_end"])
        statistics = torch.as_tensor(list(row[additional_cols].values))
        
        inputs = {
            "K_mer": kmer_seq,
            "K_mer_sparsed":kmer_seq_sparsed,
            "antivenom": antivenom,
            "genus": genus,
            "species": species,
            "proteinfam": proteinfam,
            "position_start": position_start,
            "position_end": position_end,
            "statistics": statistics
        }

        if self.is_train: 
            return inputs, torch.as_tensor([row[self.label_name]])
        return inputs

In [ ]:
train_dataset = AntivenomChallengeDataSet(
    amino_acid_map=amino_acid_map,
    antivenom_map=antivenom_map,
    genus_map=genus_map,
    species_map=species_map,
    proteinfam_map=proteinfam_map,
    data=train_split_df,
    is_train=True,
    label_name="Signal",
)

val_dataset = AntivenomChallengeDataSet(
    amino_acid_map=amino_acid_map,
    antivenom_map=antivenom_map,
    genus_map=genus_map,
    species_map=species_map,
    proteinfam_map=proteinfam_map,
    data=val_split_df,
    is_train=True,
    label_name="Signal",
)

test_dataset = AntivenomChallengeDataSet(
    amino_acid_map=amino_acid_map,
    antivenom_map=antivenom_map,
    genus_map=genus_map,
    species_map=species_map,
    proteinfam_map=proteinfam_map,
    data=test_df,
    is_train=False,
)

In [ ]:
train_dataset[0]

In [ ]:
batch_size = 64
num_workers = 4
shuffle = True
drop_last = False

Now we create our PyTorch data loaders. These combine a dataset and a sampler, and provide an iterable over the given dataset.

In [ ]:
train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=shuffle,
    num_workers=num_workers,
    drop_last=drop_last,
)

val_data_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False,  # we do not want to drop the last batch during evaluation
)

test_data_loader= DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False,
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
x, y = iter(train_data_loader).next()

print(f"K_mer shape: {x['K_mer'].shape}")
print(f"K_mer sparsed shape: {x['K_mer_sparsed'].shape}")
print(f"antivenom shape: {x['antivenom'].shape}")
print(f"genus shape: {x['genus'].shape}")
print(f"species shape: {x['species'].shape}")
print(f"proteinfam shape: {x['proteinfam'].shape}")
print(f"statistics shape: {x['statistics'].shape}")
print(f"target shape: {y.shape}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


K_mer shape: torch.Size([64, 16])
K_mer sparsed shape: torch.Size([64, 320])
antivenom shape: torch.Size([64])
genus shape: torch.Size([64])
species shape: torch.Size([64])
proteinfam shape: torch.Size([64])
statistics shape: torch.Size([64, 15])
target shape: torch.Size([64, 1])


## Define the model
For this example we will build an LSTM architeture. It is your task to come up with more performant architectures to improve the scores.

In [ ]:
class SimpleSeqModel(nn.Module):
    def __init__(
        self,
        K_mer_emb_size,
        K_mer_nunique,
        K_mer_sparsed_emb_size,
        K_mer_sparsed_nunique,
        antivenom_emb_size,
        antivenom_unique,
        genus_emb_size,
        genus_unique,
        species_emb_size,
        species_unique,
        proteinfam_emb_size,
        proteinfam_unique,
        statistics_emb_size,
        statistics_unique,
        max_Position_start,
        Position_start_emb_size,
    ): 
        super().__init__()
        self.K_mer_emb_size = K_mer_emb_size        
        self.K_mer_nunique = K_mer_nunique 
        self.K_mer_sparsed_emb_size = K_mer_sparsed_emb_size        
        self.K_mer_sparsed_nunique = K_mer_sparsed_nunique                
        self.antivenom_emb_size = antivenom_emb_size  
        self.antivenom_unique = antivenom_unique                  
        self.genus_emb_size = genus_emb_size  
        self.genus_unique = genus_unique                  
        self.species_emb_size = species_emb_size  
        self.species_unique = species_unique                  
        self.proteinfam_emb_size = proteinfam_emb_size  
        self.proteinfam_unique = proteinfam_unique                     
        self.statistics_emb_size = statistics_emb_size  
        self.statistics_unique = statistics_unique     
        
        
        self.Kmer_emb_layer = nn.Embedding(
            num_embeddings=self.K_mer_nunique,
            embedding_dim=self.K_mer_emb_size,
        )
        self.Kmer_sparsed_emb_layer = nn.Embedding(
            num_embeddings=self.K_mer_sparsed_nunique,
            embedding_dim=self.K_mer_sparsed_emb_size,
        )
        self.Antivenom_emb = nn.Embedding(
            num_embeddings=self.antivenom_unique,
            embedding_dim=self.antivenom_emb_size,
        )

        self.Genus_emb = nn.Embedding(
            num_embeddings=self.genus_unique,
            embedding_dim=self.genus_emb_size,
        )
        self.Species_emb = nn.Embedding(
            num_embeddings=self.species_unique,
            embedding_dim=self.species_emb_size,
        )
        self.ProteinFam_emb = nn.Embedding(
            num_embeddings=self.proteinfam_unique,
            embedding_dim=self.proteinfam_emb_size,
        )
        
        self.Statistics_emb = nn.Embedding(
            num_embeddings=self.statistics_unique,
            embedding_dim=self.statistics_emb_size,
        )

    
        self.Position_start_emb = nn.Embedding(
            num_embeddings=max_Position_start,
            embedding_dim=Position_start_emb_size,
        )
        self.Features = nn.Linear(
            in_features=self.antivenom_emb_size + self.genus_emb_size + self.species_emb_size + 
            self.proteinfam_emb_size + Position_start_emb_size,
            out_features=128,
        )
        
        self.Lstm_layer_1 = nn.LSTM(
            input_size=self.K_mer_emb_size,
            hidden_size=256,
            num_layers=1,
            bidirectional=True,
            batch_first=True,
        )
        self.Lstm_layer_2 = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=1,
            bidirectional=False,
            batch_first=True,
        )
        
        self.Lstm_layer_11 = nn.LSTM(
            input_size=self.statistics_emb_size,
            hidden_size=256,
            num_layers=1,
            bidirectional=True,
            batch_first=True,
        )
        
        self.Lstm_layer_21 = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=1,
            bidirectional=False,
            batch_first=True,
        )

        self.Lstm_layer_12 = nn.LSTM(
            input_size=self.K_mer_emb_size,
            hidden_size=256,
            num_layers=1,
            bidirectional=True,
            batch_first=True,
        )
        
        self.Lstm_layer_22 = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=1,
            bidirectional=False,
            batch_first=True,
        )
        
        self.Linear_1 = nn.Linear(
            in_features=self.Lstm_layer_2.hidden_size + self.Lstm_layer_21.hidden_size + self.Lstm_layer_22.hidden_size + self.Features.out_features,
            out_features=512,
        )
        
        self.relu_1 = nn.ReLU()
        self.Linear_2 = nn.Linear(
            in_features=self.Linear_1.out_features, out_features=256,
        )
        self.relu_2 = nn.ReLU()
        self.Output = nn.Linear(
            in_features=self.Linear_2.out_features, out_features=1,
        )
        
    def forward(self, inputs):
        kmer_emb = self.Kmer_emb_layer(inputs["K_mer"])
        kmer_sparsed_emb = self.Kmer_emb_layer(inputs["K_mer_sparsed"])
        antivenom_emb = self.Antivenom_emb(inputs["antivenom"])
        genus_emb = self.Genus_emb(inputs["genus"])
        species_emb = self.Species_emb(inputs["species"])
        proteinfam_emb = self.ProteinFam_emb(inputs["proteinfam"])
        statistics_emb = self.Statistics_emb(inputs["statistics"])
        position_start_emb = self.Position_start_emb(inputs["position_start"])

        emb_features = torch.cat((antivenom_emb, genus_emb, species_emb, proteinfam_emb, position_start_emb), axis=1)
        features = self.Features(emb_features)
        
        lstm_1_seq, (lstm_1_h, lstm1_c) = self.Lstm_layer_1(kmer_emb)
        lstm_2_seq, (lstm_2_h, lstm2_c) = self.Lstm_layer_2(lstm_1_seq)
        
        lstm_11_seq, (lstm_11_h, lstm11_c) = self.Lstm_layer_11(statistics_emb)
        lstm_21_seq, (lstm_21_h, lstm21_c) = self.Lstm_layer_21(lstm_11_seq)
        
        lstm_12_seq, (lstm_12_h, lstm12_c) = self.Lstm_layer_12(kmer_sparsed_emb)
        lstm_22_seq, (lstm_22_h, lstm22_c) = self.Lstm_layer_22(lstm_12_seq)

        lstm_h = torch.squeeze(lstm_2_h)
        lstm_h2 = torch.squeeze(lstm_21_h)
        lstm_h3 = torch.squeeze(lstm_22_h)
        emb = torch.cat((lstm_h, lstm_h2, lstm_h3, features), axis=1)
        linear_1 = self.relu_1(self.Linear_1(emb))
        linear_2 = self.relu_2(self.Linear_2(linear_1))
        output = self.Output(linear_2)
        return output
        
        

Now that the model architecture is defined we are goint to instantiate our model. For this we need to calculate `max_Position_start` in order to calculate the size of the embedding layer we will use to encode the start position. The maximum position that the train and test dataset can have is:


In [ ]:
max_Position_start = pd.concat([train_df[["Kmer_Position_start"]], test_df[["Kmer_Position_start"]]]).Kmer_Position_start.max()+1

print(f"Max Position_start : {max_Position_start}")


Max Position_start : 596


In [ ]:
num_epochs = 10
early_stopping = 5

model = SimpleSeqModel(
    K_mer_emb_size=512,
    K_mer_sparsed_emb_size=512,
    K_mer_nunique=len(amino_acid_map),
    antivenom_emb_size=64,
    antivenom_unique=len(antivenom_map),
    genus_emb_size=64,
    genus_unique=len(genus_map),
    species_emb_size=64,
    species_unique=len(species_map),
    proteinfam_emb_size=64,
    proteinfam_unique=len(proteinfam_map),    
    statistics_emb_size=512,
    statistics_unique=15,
    max_Position_start=max_Position_start,
    Position_start_emb_size=64,
)

loss_fn = nn.MSELoss()

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

writer = SummaryWriter()
writer.add_graph(model, {k: v.to(device) for k, v in next(iter(train_data_loader))[0].items()})

TypeError: ignored

### Training the model
We define a simple training loop


In [ ]:
def train_func(
    train_data_loader,
    val_data_loader,
    model,
    loss_fn,
    optimizer,
    num_epochs,
    device,
    early_stopping=5,
): 
    total_batches = len(train_data_loader)
    total_batches_val = len(val_data_loader)
    train_loss = []
    
    n_iter = 0
    for epoch in range(num_epochs): 
        tqdm_bar = tqdm(train_data_loader, desc=f"epoch {epoch}", position=0) 
        old_val_loss = np.inf
        wating = 0
        model.train()
        for batch_number, (X, y) in enumerate(tqdm_bar):
            y = y.type(torch.FloatTensor).to(device)
            X = {k: X[k].to(device) for k in X}
            
            optimizer.zero_grad()
            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            loss = loss.item()
            train_loss.append(loss)

            writer.add_scalar("loss/train", loss, n_iter)
            n_iter += 1

            if batch_number % 25 == 0: 
                tqdm_bar.set_postfix(
                    {
                        "train": f"{batch_number}/{total_batches} loss: {loss:.3} epoch loss: {np.mean(train_loss):.3}",
                    },
                )

        val_tqdm_bar = tqdm(
            val_data_loader, desc=f"epoch {epoch}", position=0, leave=True,
        ) 
        val_loss = []
        model.eval()
        with torch.no_grad(): 
            for batch_number, (X, y) in enumerate(val_tqdm_bar):
                y = y.type(torch.FloatTensor).to(device)
                X = {k: X[k].to(device) for k in X}
                
                pred = model(X)
                val_loss.append(loss_fn(pred, y).item())

                writer.add_scalar("loss/validation", np.random.random(), n_iter)

                if batch_number % 25 == 0: 
                    val_tqdm_bar.set_postfix(
                        {
                            "valid": f"{batch_number}/{total_batches_val} val loss: {np.mean(val_loss):.3}"
                        },
                    )
        
        new_val_loss = np.mean(val_loss)

        if new_val_loss > old_val_loss:
            wating += wating
        else:
            old_val_loss = new_val_loss

        if wating > early_stopping:
            break

In [ ]:
%tensorboard --logdir runs

In [ ]:
train_func(
    train_data_loader=train_data_loader,
    val_data_loader=val_data_loader,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    early_stopping=early_stopping,
)

In [ ]:
torch.save(model, "model.pth")

### Sample baseline Submission
Finally we will prepare a baseline submission to Zindi 


In [ ]:
def predict_test(data_loader, path): 
    model = torch.load(path).to(device)
    tqdm_bar = tqdm(data_loader, desc="Inference", position=0, leave=True) 
    total_batches = len(tqdm_bar)

    preds = []
    with torch.no_grad():
        for batch_number, X in enumerate(tqdm_bar):
            X= {k: X[k].to(device) for k in X}
            pred = model(X)
            preds.append(pred.cpu().numpy())

        preds = np.concatenate(preds)
        return preds

In [ ]:
test_pred = predict_test(test_data_loader,"model.pth")

In [ ]:
sample_submission=test_df[["ID"]]
sample_submission["Signal"] = test_pred.reshape((-1))
sample_submission.to_csv("Submissions/sample_submission_lb_ug_more_features+stats+KMER_SEQ.csv",index=False)

That is it! Now we can upload the sample_submission.csv to Zindi! As a final thing lets look at it. 

In [ ]:
sample_submission.head()

In [ ]:
sample_submission["Signal"].hist()

In [ ]:
sample_submission["Signal"].describe()